In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/dfs/scratch1/simran/tutorial/bootleg-internal/notebooks')
import os
import random
import ujson
import argparse
import pandas as pd
import numpy as np
import pickle
import json
import csv
from tqdm import tqdm
from collections import defaultdict, OrderedDict
import jsonlines
pd.set_option('display.max_colwidth', -1)

import load_entity_profiles

In [ ]:
from load_entity_profiles import es, esp, type_vocab_hy, typeid2typename_hy, type_vocab_wd, typeid2typename_wd, type_vocab_rel, typeid2typename_rel, alias_qid_count, rel_to_name
import utils_not_for_public as du

In [97]:
# es, esp, type_vocab_hy, typeid2typename_hy, type_vocab_wd, typeid2typename_wd, type_vocab_rel, typeid2typename_rel, alias_qid_count = du.load_esp("wiki",overwrite=False)


In [11]:
def load_rel_names():
    return json.load(open("/dfs/scratch1/simran/bootleg_downstream/archive/wikidata_mappings/pid_names.json"))

rel_to_name = load_rel_names()

def get_relations_lst(qid1, qid2):
    # If there are multiple relationships between two qids (both directions)
    rels = [esp.get_relation_name(r) for r in esp.get_all_relations(qid1, qid2)]
    rels = [rel_to_name.get(rel, rel) for rel in rels]
    if rels:
        return rels[0]
    else:
        return ""

In [12]:

# from bootleg.symbols.entity_symbols import EntitySymbols
# from bootleg.symbols.type_symbols import TypeSymbols
# from bootleg.symbols.kg_symbols import KGSymbols

# input_dir = '/dfs/scratch0/lorr1/bootleg/data/wiki_0906_pg/'
# emb_dir = '/dfs/scratch0/lorr1/bootleg/embs'
# entity_dump = EntitySymbols(load_dir=os.path.join(input_dir, "entity_db/entity_mappings"))
# types_hy = TypeSymbols(entity_dump, emb_dir, max_types=3, type_vocab_file="hyena_vocab.json", type_file="hyena_types_0906_pg.json")
# types_wd = TypeSymbols(entity_dump, emb_dir, max_types=3, type_vocab_file="wikidata_to_typeid_0906_pg.json", type_file="wikidata_types_0906_pg.json")
# types_rel = TypeSymbols(entity_dump, emb_dir, max_types=50, type_vocab_file="relation_to_typeid_0906_pg.json", type_file="kg_relation_types_0906_pg.json")
# kg_syms = KGSymbols(entity_dump, emb_dir, "kg_adj_0906_pg.txt")
# wdtypes2title = json.load(open(os.path.join(emb_dir, "wikidatatypeid_to_title_0906_pg.json")))
# a2q = json.load(open(os.path.join(input_dir, "entity_db/entity_mappings/alias2qids.json")))

# Utility functions

In [13]:
def load_mentions(file): 
    lines = []
    with jsonlines.open(file) as f: 
        for line in f: 
            new_line = {
                'id': line['id'],
                'sentence': line['sentence'],
                'aliases': line['aliases'], 
                'spans': line['spans'],
                'qids': line['qids'],
                #'anchor': line['anchor'],
                'sent_idx_unq': line['sent_idx_unq'],
                'probs': line['probs'], # probability of gold  ( value )
                'ctx_emb_ids': line['ctx_emb_ids'],
                'entity_ids': line['entity_ids'] 
                # candprobs # probability for all candidates (N x k matrix)
            }
            lines.append(new_line)
    return pd.DataFrame(lines)

In [14]:
def get_span(span):
    span = span.split(":")
    span_s = int(span[0])
    span_e = int(span[1])
    return span_s, span_e

In [15]:
def load_mapping(file):
    lines = []
    with jsonlines.open(file) as f: 
        for line in f: 
            new_line = {
                'sentence': line['sentence'],
                'id': line['id']
            }
            lines.append(new_line)
    return pd.DataFrame(lines)

In [16]:
def load_dataset(file):
    lines = []
    with json.open(file) as f: 
        lines.append(new_line)
    return pd.DataFrame(lines)

# Load bootleg labels, mapping of labeled sentences to tacred ids, and tacred splits

In [17]:
base_directory = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/ft_expts/best_model_101520/'
expt_directory = 'basic_full_sentences/'
boot_labels_file = "{}{}bootleg_labels.jsonl".format(base_directory, expt_directory)
bootleg_labels_df = load_mentions(boot_labels_file)
print(bootleg_labels_df.columns)
print(bootleg_labels_df.shape)

Index(['id', 'sentence', 'aliases', 'spans', 'qids', 'sent_idx_unq', 'probs',
       'ctx_emb_ids', 'entity_ids'],
      dtype='object')
(106264, 9)


In [18]:
# all_qids = []
# for ind, row in bootleg_labels_df.iterrows():
#     all_qids.extend(row['qids'])
    
# print(len(set(all_qids)))

# with open('../bootleg_092620/tacred_qids_bootleg_full_0926.txt', 'w') as f:
#     for item in set(all_qids):
#         f.write("%s\n" % item)

In [19]:
base_data = '{}{}/'.format(base_directory, expt_directory)
out_dir = '{}{}/'.format(base_directory, expt_directory)

In [149]:
train_file = "{}/train_ent.json".format(base_data)
with open(train_file) as train:
    df_train = json.load(train)
    df_train = pd.DataFrame.from_dict(df_train, orient='columns')
    print(df_train.shape)
    
dev_file = "{}/dev_rev_ent.json".format(base_data)
with open(dev_file) as dev:
    df_dev = json.load(dev)
    df_dev = pd.DataFrame.from_dict(df_dev, orient='columns')
    print(df_dev.shape)
    
test_file = "{}/test_rev_ent.json".format(base_data)
with open(test_file) as test:
    df_test = json.load(test)
    df_test = pd.DataFrame.from_dict(df_test, orient='columns')
    print(df_test.shape)


(68124, 20)
(22631, 20)
(15509, 20)


In [150]:
train_ids = df_train.loc[:,'id'].tolist()
dev_ids = df_dev.loc[:,'id'].tolist()
test_ids = df_test.loc[:,'id'].tolist()

In [151]:
df_train.columns

Index(['id', 'docid', 'relation', 'token', 'subj_start', 'subj_end',
       'obj_start', 'obj_end', 'subj_type', 'obj_type', 'stanford_pos',
       'stanford_ner', 'stanford_head', 'stanford_deprel', 'entity_emb_id',
       'entity_emb_id_first', 'ent_id', 'ent_id_first', 'static_ent_emb_id',
       'static_ent_emb_id_first'],
      dtype='object')

# Load rel2id map

In [152]:
model_base = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_iclr_model/bootleg_downloads/emb_data/'
rel2id = '{}/relation_to_typeid_0905.json'.format(model_base)
typeqid_to_id = json.load(open("{}wikidata_to_typeid_0905.json".format(model_base)))
wikidata_types_wiki_filt_to_id = json.load(open('{}wikidata_types_0905.json'.format(model_base)))
with open(rel2id) as f: 
    relation_to_typeid = ujson.load(f)
    
relation_to_typeid['founded by']

124

In [153]:
print(relation_to_typeid['founded by'])

124


In [154]:
# rel2id = '/dfs/scratch1/mleszczy/contextual-embeddings-git/embs/relation_to_typeid_adj.json'
# with open(rel2id) as f: 
#     relation_to_typeid = ujson.load(f)
# relation_to_typeid['founder']  

In [155]:
rel_to_name = load_rel_names()

# Functions to generate the features
- Choice #1: Only add a relation embedding to the subject entity because these relations are not symmetric
- Choice #2: Take the first relation/type id in the returned list of relations/types for the input entities
- Choice #3: I made the subj and obj type entities part of the same "feature" list for an example rather than independent. This pertains to the entity embedding features too! (THINK ABOUT THIS)

In [156]:
all_relations = []
all_types = []


In [157]:
def get_rel_dict(df):
    # add a feature for relation embeddings
    rel_dict = {}
    
    rel_names_lst = []
    rel_exits_count = 0
    not_in_map_count = 0
    not_in_map_list = []
    rels_added = []
    for ind, row in df.iterrows():
        idx = row['id']
        qids = row['ent_id']
        ss, se = row['subj_start'], row['subj_end']
        subj_qids = qids[ss:se+1]
        os, oe = row['obj_start'], row['obj_end']
        obj_qids = qids[os:oe+1]
        length = len(qids)

        rel = get_relations_lst(subj_qids[0], obj_qids[0])
#         if rel == 'founded by':
#             rel = 'founder'
#         elif rel == 'political party':
#             rel = 'political alignment'
#         elif rel == 'subsidiary':
#             rel = 'subsidiaries'
#         elif rel == 'director / manager':
#             rel = "manager/director"
#         elif rel == 'parent organization':   
#             rel = 'parent company'
#         elif rel == 'languages spoken, written or signed':
#             rel = 'languages spoken or published'
#         elif rel == 'language of work or name':
#             rel = 'language of work (or name)'
#         elif rel == 'commander of (DEPRECATED)':    
#             rel = 'commander of'
#         elif rel == 'original broadcaster':
#             rel = 'broadcast by'

        # add embedding index to dict
        rel_emb_id_result = [-1] * length

        if rel and rel in relation_to_typeid:
            all_relations.append(rel)
            rel_id = relation_to_typeid[rel]+1
            rel_emb_id_result[ss] = rel_id
            rel_dict[idx] = rel_emb_id_result
            rel_exits_count += 1
            rels_added.append(rel)
        elif rel not in relation_to_typeid:
            rel_emb_id_result[ss] = 0
            not_in_map_count += 1  
            not_in_map_list.append(rel)
        rel_names_lst.append(rel)
        rel_dict[idx] = rel_emb_id_result

    print("The number of pairs for which a rel exists is: ", rel_exits_count)
    print("The number of pairs for which a rel does not exist is: ", not_in_map_count)
    print(Counter(rels_added).most_common(100))
    return rel_dict, rel_names_lst, not_in_map_list

In [158]:
def load_json_file(filename):
    with open(filename, 'r') as f:
        contents = ujson.load(f)
    return contents

In [159]:
typename2title_wd = load_json_file("{}wikidatatypeid_to_title_0905.json".format(model_base))
def get_type_ids(df):
    type_dict = {}
    
    # analysis structures
    exists_subj = 0
    subj_not_mapped = 0
    exists_obj = 0
    obj_not_mapped = 0
    not_exists = 0
    not_in_map_list = []
    
    
    for ind, row in df.iterrows():
        idx = row['id']
        qids = row['ent_id']
        ss, se = row['subj_start'], row['subj_end']
        subj_qids = qids[ss:se+1]
        os, oe = row['obj_start'], row['obj_end']
        obj_qids = qids[os:oe+1]
        length = len(qids)
                     
        # get the first type in the types list (TODO, better way to do this in the future!)    
        type_emb_id_result = [-1] * length
        try:
            if subj_qids[0] and subj_qids[0] != 'UNK':
                if subj_qids[0] in wikidata_types_wiki_filt_to_id:
                    type_subj_id_lst = wikidata_types_wiki_filt_to_id[subj_qids[0]]
                    types_subj = du.get_qid_types(subj_qids[0], esp, typeid2typename_wd)
                    if type_subj_id_lst:
                        all_types.extend(types_subj)
                        type_emb_id_result[ss] = type_subj_id_lst[0]+1
                        exists_subj +=1
                    else:
                        type_emb_id_result[ss] = 0
                        not_exists += 1
                else:
                    subj_not_mapped += 1
                    # print(subj_qids[0])

            if obj_qids[0] and obj_qids[0] != 'UNK':
                if obj_qids[0] in wikidata_types_wiki_filt_to_id:
                    type_obj_id_lst = wikidata_types_wiki_filt_to_id[obj_qids[0]]
                    types_obj = du.get_qid_types(obj_qids[0], esp, typeid2typename_wd)
                    if type_obj_id_lst:
                        all_types.extend(types_obj)
                        type_emb_id_result[os] = type_obj_id_lst[0]+1
                        exists_obj +=1
                    else:
                        type_emb_id_result[os] = 0
                        not_exists += 1
                else:
                    obj_not_mapped += 1
                    # print(obj_qids[0])
        except:
            pass
                
        type_dict[idx] = type_emb_id_result

    print("The number of subjects with a type relation: ", exists_subj)
    print("The number of objects with a type relation: ",exists_obj)
    
    
    return type_dict

In [160]:
def get_all_types_in_sentence(df):
    # analysis structures
    type_dict = {}
    exists_subj = 0
    subj_not_mapped = 0
    exists_obj = 0
    obj_not_mapped = 0
    not_exists = 0
    not_in_map_list = []
    
    
    for ind, row in df.iterrows():
        idx = row['id']
        qids = row['ent_id']
        ss, se = row['subj_start'], row['subj_end']
        subj_qids = qids[ss:se+1]
        os, oe = row['obj_start'], row['obj_end']
        obj_qids = qids[os:oe+1]
        length = len(qids)
                     
        # get the first type in the types list (TODO, better way to do this in the future!)    
        try:
            type_emb_id_result = [[]] * length
            for i in range(len(qids)):
                qid = qids[i]           
                if qid and qid != 'UNK':
                    if qid in wikidata_types_wiki_filt_to_id:
                        type_id_lst = wikidata_types_wiki_filt_to_id[qid]
                        types = du.get_qid_types(qid, esp, typeid2typename_wd)
                        if type_id_lst:
                            all_types.extend(types)
                            type_emb_id_result[i] = [type_id+1 for type_id in type_id_lst]
        except:
            pass
        type_dict[idx] = type_emb_id_result
        
        
    return type_dict  

In [161]:
get_all_types_in_sentence(df_train)
get_all_types_in_sentence(df_dev)
get_all_types_in_sentence(df_test)

{'098f665fb966708cfcd2': [[],
  [],
  [],
  [],
  [],
  [],
  [184, 357, 1140],
  [],
  [],
  [],
  [3850, 6819, 7132, 12286, 22558, 22559],
  [12, 24, 554],
  [12, 24, 554],
  [12, 24, 554],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [17501],
  [17501],
  [17501],
  [],
  [469, 3754],
  [12],
  [12],
  [],
  [],
  [],
  [],
  [],
  [270],
  [270],
  [270],
  [270],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [395],
  [],
  [],
  [411, 2841],
  [411, 2841],
  [150],
  [150],
  [],
  [75],
  [75],
  []],
 '098f665fb90bef0c4ca4': [[1565, 3775, 6526],
  [2222],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [1565, 3775, 6526],
  [],
  [],
  [4130],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [4130],
  [4130],
  [],
  [],
  [],
  [],
  [],
  [],
  [2222],
  [],
  [],
  [],
  []],
 '098f665fb9ef7dbc81e7': [[12, 24, 57, 100, 116, 257],
  [],
  [],
  [],
  [232, 3850, 6819, 7035, 7138, 22562],
  [],
  [],
  [],
  [5119],
  [],
  [],
  [],
  [14],
  [121],
  [],
  [3850, 

# Add the three features to the tacred data

In [162]:
bootleg_to_tacred_type_dict = {}

# create this and also stop taking just the first type/rel!!!

### Training Data

In [163]:
df_train.shape

(68124, 20)

In [164]:
# add a feature for relation embeddings
from collections import Counter
rel_dict, rel_names_lst, not_in_map_list = get_rel_dict(df_train)
print("\nHere's an example entry in the relation dict.")
for k, v in rel_dict.items():
    print(k, v)
    break 

The number of pairs for which a rel exists is:  3913
The number of pairs for which a rel does not exist is:  64211
[('country', 552), ('country of citizenship', 429), ('political party', 199), ('member of', 187), ('position held', 170), ('headquarters location', 159), ('located in the administrative territorial entity', 140), ('parent organization', 128), ('member of sports team', 121), ('employer', 118), ('place of birth', 116), ('occupation', 101), ('founded by', 94), ('part of', 92), ('officeholder', 70), ('place of death', 70), ('spouse', 59), ('cast member', 53), ('cause of death', 40), ('director / manager', 40), ('sibling', 39), ('military branch', 36), ('head of state', 33), ('applies to jurisdiction', 29), ('father', 29), ('owned by', 27), ('educated at', 26), ('instance of', 25), ('operator', 24), ('location of formation', 24), ('diplomatic relation', 24), ('replaces', 23), ('conflict', 23), ('organizer', 23), ('capital of', 21), ('chairperson', 21), ('league', 20), ('politic

In [165]:
from collections import Counter
Counter(not_in_map_list).most_common(100)

[('', 64211)]

In [166]:
# add a feature for type embeddings
type_dict = get_type_ids(df_train)
print("\nHere's an example of the type feature")
for k, v in type_dict.items():
    print(k, v)
    break 

The number of subjects with a type relation:  51373
The number of objects with a type relation:  43059

Here's an example of the type feature
61b3a5c8c9a882dcfcd2 [12, -1, -1, -1, -1, -1, -1, -1, -1, -1, 283, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [167]:
type_dict_lst = get_all_types_in_sentence(df_train)
print("\nHere's an example of the type feature")
for k, v in type_dict_lst.items():
    print(k, v)
    break 


Here's an example of the type feature
61b3a5c8c9a882dcfcd2 [[12], [12], [], [], [], [], [], [], [], [], [283], [283], [283], [], [3238, 3582, 4918, 10722], [], [], [20002], [20002], [20002], [], [], [184, 5552], [184, 5552], [184, 5552], [], [], [3809, 8788], [3809, 8788], [1833], [1833], [1833], [], [22449, 23783], [22449, 23783], [], [], [], [98], [98], []]


In [168]:
df_train['rel_emb_id'] = df_train['id'].map(rel_dict)
df_train['type_emb_id'] = df_train['id'].map(type_dict)
#df_train['type_emb_id_lst'] = df_train['id'].map(type_dict_lst)

In [169]:
df_train.shape

(68124, 22)

In [170]:
print(len(set(all_relations)))
print(len(set(all_types)))

129
10398


In [171]:
# with open('../bootleg_092620/tacred_types_bootleg_full_0926.txt', 'w') as f:
#     for item in set(all_types):
#         f.write("%s\n" % item)

In [172]:
# names = []
# for tup in subj_type_list:
#     try:
#         title = es.get_title(tup)
#         names.append(title)
#     except:
#         pass
# names = Counter(names).most_common(200) 
# print(names)

# Dev Data

In [173]:
df_dev.shape

(22631, 20)

In [174]:
# add a feature for relation embeddings
rel_dict, rel_names_lst, not_in_map_list = get_rel_dict(df_dev)

print("\nHere's an example entry in the relation dict.")
for k, v in rel_dict.items():
    print(k, v)
    break 

The number of pairs for which a rel exists is:  1412
The number of pairs for which a rel does not exist is:  21219
[('country', 274), ('country of citizenship', 213), ('place of death', 113), ('headquarters location', 74), ('founded by', 58), ('place of birth', 52), ('owner of', 48), ('occupation', 41), ('spouse', 40), ('parent organization', 39), ('cause of death', 36), ('country for sport', 36), ('member of', 28), ('religion', 26), ('member of sports team', 25), ('notable work', 23), ('political ideology', 20), ('chairperson', 18), ('head of government', 17), ('political party', 15), ('employer', 14), ('officeholder', 14), ('contains administrative territorial entity', 11), ('subsidiary', 11), ('location of formation', 11), ('director / manager', 10), ('participant in', 8), ('director', 7), ('unmarried partner', 7), ('head of state', 7), ('allegiance', 7), ('capital of', 7), ('languages spoken, written or signed', 6), ('designated as terrorist by', 6), ('movement', 6), ('educated at'

In [175]:
# add a feature for type embeddings
type_dict = get_type_ids(df_dev)

print("\nHere's an example of the type feature")
for k, v in type_dict.items():
    print(k, v)
    break 

The number of subjects with a type relation:  15276
The number of objects with a type relation:  13758

Here's an example of the type feature
e7798fb926b9403cfcd2 [-1, -1, -1, -1, -1, -1, -1, -1, 105, -1, -1, -1, 184, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [176]:
type_dict_lst = get_all_types_in_sentence(df_dev)
print("\nHere's an example of the type feature")
for k, v in type_dict_lst.items():
    print(k, v)
    break 


Here's an example of the type feature
e7798fb926b9403cfcd2 [[], [], [], [], [], [258, 1865, 4971], [258, 1865, 4971], [258, 1865, 4971], [105, 301, 410], [105, 301, 410], [], [], [184, 1098, 4971, 8596], [], [], [12, 105, 1148], [12, 105, 1148], [], [], [], [], [], [], [], [], []]


In [177]:
df_dev['rel_emb_id'] = df_dev['id'].map(rel_dict)
df_dev['type_emb_id'] = df_dev['id'].map(type_dict)
#df_dev['type_emb_id_lst'] = df_dev['id'].map(type_dict_lst)

In [178]:
# print(df_dev[df_dev['id'].str.contains('e7798fb926b9403cfcd2')])
# typeid2typename_wd[232]

In [179]:
df_dev.shape

(22631, 22)

# Test Data

In [180]:
df_test.shape

(15509, 20)

In [181]:
# add a feature for relation embeddings
rel_dict, rel_names_lst, not_in_map_list = get_rel_dict(df_test)

print("\nHere's an example entry in the relation dict.")
for k, v in rel_dict.items():
    print(k, v)
    break 

The number of pairs for which a rel exists is:  477
The number of pairs for which a rel does not exist is:  15032
[('country of citizenship', 109), ('country', 85), ('place of birth', 53), ('contains administrative territorial entity', 24), ('located in the administrative territorial entity', 24), ('parent organization', 22), ('operating area', 22), ('chairperson', 19), ('place of death', 14), ('spouse', 14), ('headquarters location', 12), ('occupation', 11), ('employer', 7), ('parent taxon', 6), ('educated at', 6), ('founded by', 4), ('head of state', 4), ('country for sport', 4), ('work location', 3), ('filming location', 3), ('part of', 3), ('territory claimed by', 3), ('named after', 3), ('member of', 3), ('main subject', 2), ('political party', 2), ('family', 2), ('residence', 2), ('head of government', 2), ('country of origin', 2), ('languages spoken, written or signed', 2), ('position held', 1), ('director / manager', 1), ('religion', 1), ('applies to jurisdiction', 1), ('ethnic

In [182]:
# add a feature for type embeddings
type_dict = get_type_ids(df_test)

print("\nHere's an example of the type feature")
for k, v in type_dict.items():
    print(k, v)
    break 

The number of subjects with a type relation:  8313
The number of objects with a type relation:  8850

Here's an example of the type feature
098f665fb966708cfcd2 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 270, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [183]:
type_dict_lst = get_all_types_in_sentence(df_test)
print("\nHere's an example of the type feature")
for k, v in type_dict_lst.items():
    print(k, v)
    break 


Here's an example of the type feature
098f665fb966708cfcd2 [[], [], [], [], [], [], [184, 357, 1140], [], [], [], [3850, 6819, 7132, 12286, 22558, 22559], [12, 24, 554], [12, 24, 554], [12, 24, 554], [], [], [], [], [], [], [], [17501], [17501], [17501], [], [469, 3754], [12], [12], [], [], [], [], [], [270], [270], [270], [270], [], [], [], [], [], [], [], [], [395], [], [], [411, 2841], [411, 2841], [150], [150], [], [75], [75], []]


In [184]:
df_test['rel_emb_id'] = df_test['id'].map(rel_dict)
df_test['type_emb_id'] = df_test['id'].map(type_dict)
#df_test['type_emb_id_lst'] = df_test['id'].map(type_dict_lst)

In [185]:
df_test.shape

(15509, 22)

In [186]:
df_test.columns

Index(['obj_type', 'docid', 'subj_type', 'subj_end', 'stanford_pos',
       'stanford_head', 'subj_start', 'token', 'obj_start', 'relation',
       'stanford_deprel', 'stanford_ner', 'id', 'obj_end', 'entity_emb_id',
       'entity_emb_id_first', 'ent_id', 'ent_id_first', 'static_ent_emb_id',
       'static_ent_emb_id_first', 'rel_emb_id', 'type_emb_id'],
      dtype='object')

# Save the appended data

In [191]:
print(out_dir)
out_dir = '/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/ft_expts/best_model_101520/basic_full_sentences/typerel/'

/dfs/scratch1/simran/tacred/tacred-relation-bootleg/dataset_bootleg_ablations/ft_expts/best_model_101520/basic_full_sentences/typerel/


In [192]:
train_out = df_train.to_json(r'{}/train_ent.json'.format(out_dir),orient='records')

In [193]:
dev_out = df_dev.to_json(r'{}/dev_rev_ent.json'.format(out_dir),orient='records')

In [194]:
test_out = df_test.to_json(r'{}/test_rev_ent.json'.format(out_dir),orient='records')